# 该模型模拟了细菌菌落在平坦表面上的生长与扩展过程
此流程再现了主文本中模拟群体遗传噪声如何导致同样适应度的扩张菌落空间分隔的主要实验方案。

In [ ]:
import cobra
import cobra.io # for the ijo1366 model
import sys
import copy
import numpy as np
#sys.path.append("/home/jeremy/Dropbox/work_related/harcombe_lab/segre/cometspy") # not necessary if you pip install cometspy
import cometspy as c

我们将使用E. coli core模型

In [ ]:
ecoli = cobra.io.load_model('textbook')

我们现在将把这个模型转换为COMETS模型，并设置两个参数选项：一个是反映群体遗传噪声的参数，被称为“中性漂变”（neutral drift）；另一个是基于对流运动的参数。

In [ ]:
grid_size = 40
ecoli_comets = c.model(ecoli)
ecoli_comets.reactions.loc[ecoli_comets.reactions.EXCH, "LB"] = -1000
ecoli_comets.add_nonlinear_diffusion_parameters(convNonlinDiffZero = 0.,
                                    convNonlinDiffN = 6.e-6,
                                    convNonlinDiffExponent = 1.0,
                                    convNonlinDiffHillN = 2.,
                                    convNonlinDiffHillK = 0.001)
ecoli_comets.add_neutral_drift_parameter(0.01)

让我们精确复制 ecoli comets模型。

In [ ]:
import random

ecoli_comets2 = copy.copy(ecoli_comets)
ecoli_comets2.id = "e_coli_core2"

ecoli_comets.initial_pop = [[x, 0, np.random.normal(6.4e-5, 1e-6)] for x in range(grid_size)]
ecoli_comets2.initial_pop = [[x, 0, np.random.normal(6.4e-5, 1e-6)] for x in range(grid_size)]

这次模拟将在一个100x100的网格上构建一个位于中心的单一菌落。碳源将在整个区域内均匀分布，并且在网格的一端设定为固定的初始浓度。

In [ ]:
ly = c.layout([ecoli_comets, ecoli_comets2])
ly.grid = [grid_size, grid_size]
ly.set_specific_metabolite("glc__D_e", 2.2e-3)
ly.set_specific_metabolite("h2o_e", 1000)
ly.set_specific_metabolite("nh4_e", 1000)
ly.set_specific_metabolite("o2_e", 1000)
ly.set_specific_metabolite("pi_e", 1000)
ly.set_specific_metabolite("h_e", 1000)

for x in range(grid_size):
    ly.set_specific_static_at_location("glc__D_e", (x, grid_size-1), 1.1e-3)

我们需要设置的主要参数是 biomassmotionstyle，必须设置为“ConvNonlin Diffusion 2D”。然后，为了捕获空间信息，我们还将记录生物量（而不仅仅是总生物量）。最后，我们还将调整一些其他参数。这些存储在字典all_params中。

In [ ]:
p = c.params()

cycles = 1000

p.all_params["biomassMotionStyle"] = "ConvNonlin Diffusion 2D"
p.all_params["writeBiomassLog"] = True
p.all_params["BiomassLogRate"] = cycles
p.all_params["maxCycles"] = cycles
p.all_params["timeStep"] = 0.05
p.all_params["spaceWidth"] = 0.01
p.all_params["defaultDiffConst"] = 6.e-6
p.all_params["maxSpaceBiomass"] = 1000000
p.all_params["minSpaceBiomass"] = 1.0e-9
p.all_params["allowCellOverlap"] = True
p.all_params["growthDiffRate"] = 0
p.all_params["defaultVmax"] = 18.5
p.all_params["defaultKm"] = 0.000015
p.all_params["defaultHill"] = 1
p.all_params["exchangestyle"] = "Monod Style"

现在我们制作一个模拟对象并运行它。这可能需要一段时间。

In [ ]:
sim = c.comets(ly, p)
sim.run()

现在让我们绘制结果图。我们使用 get_biomass_image 辅助函数，然后使用 matplotlib 进行绘图。

In [ ]:
im = sim.get_biomass_image('e_coli_core', cycles).astype(float)
im = im / np.max(im)
im2 = sim.get_biomass_image('e_coli_core2',cycles).astype(float)
im2 = im2 / np.max(im2)
final = np.zeros((grid_size,grid_size,3))
final[:,:,1] = im
final[:,:,2] = im2
from matplotlib import pyplot as plt
plt.imshow(final)